In [1]:
from pathlib import Path
import pandas as pd
import random

from rasa.shared.nlu.training_data.formats.rasa_yaml import RasaYAMLReader
from rasa.shared.nlu.training_data.training_data import TrainingData
from rasa.shared.nlu.training_data.entities_parser import parse_training_example

### Load Entities

In [2]:
entities = pd.read_csv('.data/entities.csv')
docs = entities[entities['type'] == 'doctor']['label'].values
services = entities[entities['type'] == 'service']['label'].values

### Helpers

In [3]:
def parse_examples(examples):
    return''.join([f'- {e}\n' for e in examples])

In [4]:
def parse_entity(value, label):
    return f'[{value}]({label})'

def parse_entities(values, label):
    return [parse_entity(value, label) for value in values]

In [5]:
def generate_sents(sent, parts, sents):
    if len(parts) == 1:
        for part in parts[0]:
            sents.append(f'{sent} {part}'.strip())            
    else:
        for part in parts[0]:
            generate_sents(f'{sent} {part}', parts[1:], sents)

### Conjugate verbs

In [6]:
from bs4 import BeautifulSoup
import requests
import re

mode_temps = ['Indicative_Present', 'Indicative_Imperfect', 'Conditional_Present', 'Conditional_First past']
pronoms = [0, 2, 3, 5]

def is_bloc(tag):
    return tag.has_attr('class') and ('modeBloc' in tag.attrs['class'] or 'conjugBloc' in tag.attrs['class'])

def get_verb_conjugate(verb):
    sents = []
    res = requests.get(f'https://leconjugueur.lefigaro.fr/french/verb/{verb}.html')
    print(f'{verb} : {res.status_code}')
    soup = BeautifulSoup(res.text)
    mode = ''
    for tag in soup.find_all(is_bloc):
        if 'modeBloc' in tag.attrs['class']:
            mode = tag.text
        else:
            temps = tag.find('div', attrs={'class':'tempsBloc'})
            if temps and f'{mode}_{temps.text}' in mode_temps:
                text = list(tag.children)[1]
                text = re.sub(r'<\/?(b|p)>', '', str(text))
                sents += [sent for i, sent in enumerate(text.split('<br/>')) if i in pronoms]
    return sents
            

In [7]:
infinitive_verbs = ['demander', 'vouloir', 'reclamer', 'aimer', 'desirer', 'aspirer', 'esperer', 'exiger', 'souhaiter', 'rever']

verbs = {}
for verb in infinitive_verbs:
    verbs[verb] = get_verb_conjugate(verb)

demander : 200
vouloir : 200
reclamer : 200
aimer : 200
desirer : 200
aspirer : 200
esperer : 200
exiger : 200
souhaiter : 200
rever : 200


### Generate doctor sents

In [8]:
wants = []
for verb in ['vouloir', 'aimer', 'desirer', 'aspirer', 'souhaiter', 'esperer']:
    wants += verbs[verb]

for verb in ['reclamer', 'exiger', 'rever']:
    wants += [f'{v} de' for v in verbs[verb]]
    
for verb in ['demander']:
    wants += [f'{v} à' for v in verbs[verb]]
    

contacts = ['parler', 'causer', 'telephoner'] 
wichs = ['avec le docteur', 'avec la docteur', 'au docteur', 
 'avec le médecin', 'avec la médecin', 'au médecin',
 'avec Mr', 'avec Mme', 'à Monsieur', 'à Madame',
 'à Mr', 'à Mme', 'à Monsieur', 'à Madame']
targets = parse_entities([doc.split()[0] for doc in docs] + [doc for doc in docs], 'doctor')

In [9]:
doctor_sents = []
generate_sents('', [wants, contacts, wichs, targets], doctor_sents)
len(doctor_sents)

403200

In [10]:
rsents = list(doctor_sents)
random.shuffle(rsents)
rsents[:25]

['il exigerait de causer à Madame [AL MOUSSARIH Abdallah](doctor)',
 'ils souhaitent telephoner au médecin [BOULET Stéphane](doctor)',
 'nous exigeons de telephoner à Madame [BOUCHAREB](doctor)',
 'ils auraient aimé causer avec la docteur [ARISTIZABAL Patrick](doctor)',
 "j'aurais rêvé de causer avec le médecin [ABOAB](doctor)",
 'ils auraient demandé à parler avec Mme [ALLARD](doctor)',
 'ils auraient aimé telephoner avec Mr [BRYON](doctor)',
 'ils souhaiteraient parler à Madame [BENSAID](doctor)',
 'ils auraient aspiré parler avec Mr [BELMAGHNI Fatma](doctor)',
 'nous exigerions de telephoner à Monsieur [BELDA Eric](doctor)',
 'nous réclamons de telephoner avec la médecin [BRUN Magali](doctor)',
 "j'aurais voulu telephoner avec le docteur [BENOUAICHE Laurence](doctor)",
 'il exigerait de parler avec le docteur [BENOSMAN Hedi](doctor)',
 'ils auraient aimé parler avec Mme [ARISTIZABAL](doctor)',
 'ils auraient réclamé de causer à Madame [BLIN](doctor)',
 'ils auraient réclamé de telep

### Generate service sents

In [11]:
wichs = ['avec le service', 'au service', 
         'avec le secrétariat', 'au secrétariat', 
         'avec le secrétariat du service', 'au secrétariat du service', 
         'avec l\'acceuil', 'à l\'acceuil'
         'avec l\'acceuil du service', 'à l\'acceuil du service', 
        ]
targets = parse_entities([service.lower() for service in services], 'service')

In [12]:
service_sents = []
generate_sents('', [wants, contacts, wichs, targets], service_sents)
len(service_sents)

129600

In [13]:
rsents = list(service_sents)
random.shuffle(rsents)
rsents[:25]

['ils auraient exigé de causer avec le service [rythmologie](service)',
 "nous voulions telephoner à l'acceuil du service [gastro-entérologie](service)",
 'il désire parler au secrétariat du service [unité de soins continus](service)',
 'il voudrait causer avec le secrétariat du service [rhumatologie](service)',
 'nous aurions aimé parler au service [angiographie](service)',
 "j'aurais demandé à causer au secrétariat [assistante hôtelière](service)",
 "il veut causer à l'acceuilavec l'acceuil du service [soins ambulatoire](service)",
 'je voulais causer avec le secrétariat du service [urgence](service)',
 'il réclamait de parler au secrétariat [biomédical](service)',
 'il aimait parler au service [biomédical](service)',
 'il aurait aspiré telephoner avec le service [unité de soins continus](service)',
 'il souhaite causer au secrétariat du service [chirurgie viscérale](service)',
 "ils souhaitaient causer à l'acceuil du service [ressources humaines](service)",
 'nous aurions désiré cau

### Make yaml files

In [14]:
data = RasaYAMLReader().reads(Path('.data/nlu.yml').read_text('utf-8'))
data.intent_examples

C:\Users\pierr\Documents\projets\nt-challenge\venv\lib\site-packages\rasa\shared\utils\io.py:99: UserWarning: Training data file  has a lower format version than your Rasa Open Source installation: 3.0 < 3.1. Rasa Open Source will read the file as a version 3.1 file. Please update your version key to 3.1. See https://rasa.com/docs/rasa/training-data-format.


In [15]:
data.lookup_tables[0]['elements'] = list(services)

In [16]:
data.lookup_tables[1]['elements'] = list([doc.split()[0] for doc in docs] + [doc for doc in docs])

In [17]:
data.training_examples.clear()

In [18]:
service_rsents = list(service_sents)
random.shuffle(service_rsents)

In [19]:
for sent in service_rsents[:10000]:
    data.training_examples.append(parse_training_example(sent, 'contact_service'))

In [20]:
doctor_rsents = list(doctor_sents)
random.shuffle(doctor_rsents)

In [21]:
for sent in doctor_rsents[:10000]:
    data.training_examples.append(parse_training_example(sent, 'contact_doctor'))

In [22]:
data.persist_nlu('data/nlu_0.yml')